In [1]:
import pandas as pd
import gensim

In [2]:
IN_DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'
OUT_DATA_DIR = "../../data/processed/"

In [3]:
df = pd.read_parquet(IN_DATA_DIR + INPUT_FILE_NAME)
df.head()
len(df)

2386

In [4]:
tags = df['tags'].str.replace(', ', ',').str.lower().str.strip()
split_tags = tags.str.split(',')
tag_counts_per_talk = split_tags.apply(len)

joined_tags = tags.str.cat(sep=',').split(',')
all_tags_w_dup = pd.Series(joined_tags)

tag_counts = all_tags_w_dup.value_counts()
tag_list = list(tag_counts.index)

In [5]:
tag_cutoff = int(0.02*len(df.index))

squashed_tags = pd.DataFrame(tag_counts)
squashed_tags = squashed_tags[(squashed_tags[0]>tag_cutoff)]
squash_list = list(squashed_tags.index.values)


In [6]:
#remove ted tags
ted_tags=[]
for tag in squash_list:
    if 'ted' in tag:
        ted_tags.append(tag)
print(ted_tags)
squashed_tags = squashed_tags.drop(labels = ted_tags)
squash_list = list(squashed_tags.index.values)

['tedx', 'ted fellows', 'ted brain trust']


In [7]:
squashed_tags
print(squash_list)

['technology', 'science', 'global issues', 'culture', 'design', 'business', 'entertainment', 'health', 'innovation', 'education', 'art', 'society', 'social change', 'communication', 'politics', 'future', 'creativity', 'biology', 'humanity', 'collaboration', 'environment', 'economics', 'medicine', 'brain', 'activism', 'community', 'invention', 'children', 'history', 'health care', 'music', 'women', 'cities', 'storytelling', 'war', 'animals', 'engineering', 'leadership', 'identity', 'nature', 'computers', 'psychology', 'humor', 'life', 'performance', 'exploration', 'africa', 'data', 'photography', 'medical research', 'inequality', 'personal growth', 'neuroscience', 'government', 'climate change', 'visualizations', 'internet', 'architecture', 'sustainability', 'oceans', 'disease', 'green', 'happiness', 'biotech', 'potential', 'work', 'physics', 'media', 'film', 'violence', 'mind', 'evolution', 'big problems', 'writing', 'motivation', 'entrepreneur', 'biodiversity', 'live music', 'philosop

In [8]:
def squashing (x,squash_list):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [9]:
df1 = df.copy()
df1['squash_tags'] = df1['tags']
df1['squash_tags'] = df1['squash_tags'].map(lambda x: squashing(x,squash_list))
df1 = df1[df1['squash_tags']!='']

In [10]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [11]:
nlp_list = []
for i in range(len(squash_list)):
    nlp_list.append(nlp(squash_list[i]))
# print(nlp_list)

In [12]:
simlist = []
tagsimcount = [0]*len(squash_list)
tagsimdict = {el:[] for el in squash_list}
for outer in range(len(nlp_list)):
    for inner in range(len(nlp_list)):
        if inner<outer:
            sim = nlp_list[outer].similarity(nlp_list[inner])
            if sim > 0.65:
                ow = squash_list[outer]
                iw = squash_list[inner]
                ow_count = tag_counts.at[ow]
                iw_count = tag_counts.at[iw]
                simlist.append([ow,iw,sim])
                tagsimcount[outer] += 1
                tagsimcount[inner] += 1
                tagsimdict[ow].append(iw)
                tagsimdict[iw].append(ow)
                
items =list(tagsimdict.items())

for k,v in items:
    if v == []:
        del tagsimdict[k]             

# print(simlist)
# print(tagsimcount)
# print(tagsimdict)
sim_tags = list(tagsimdict.keys())

In [13]:
# replace tags
# GENERALISE: 'education','teaching' -> 'education'
# REMAIN: 'africa', 'asia' -> 'africa', 'asia'

#anything with ted remove hahaha
print(tagsimcount)
print(tagsimdict)

[1, 4, 3, 1, 0, 0, 0, 3, 3, 0, 1, 2, 5, 0, 0, 1, 1, 6, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 0, 0, 0, 0, 1, 5, 1, 2, 3, 1, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 4, 1, 4, 0, 0, 0, 0, 0, 5, 0, 0, 2, 0, 0, 0, 1, 2, 0, 1, 0, 0]
{'technology': ['innovation'], 'science': ['biology', 'psychology', 'physics', 'cognitive science'], 'global issues': ['social change', 'climate change', 'global development'], 'culture': ['society'], 'health': ['health care', 'medical research', 'mental health'], 'innovation': ['technology', 'creativity', 'global development'], 'art': ['photography'], 'society': ['culture', 'social change'], 'social change': ['global issues', 'society', 'personal growth', 'climate change', 'global development'], 'future': ['potential'], 'creativity': ['innovation'], 'biology': ['science', 'neuroscience', 'physics', 'genetics', 'cognitive science', 'ecology'], 'medicine': ['medical research'], 'health care'

In [14]:
def sim_counts(a):
#     print(a)
    a_count=tag_counts.at[a]
    all_tags = [a]
    all_count = [a_count]
    b = tagsimdict[a]
    for word in b:
        b_count=tag_counts.at[word]
        all_tags.append(word)
        all_count.append(b_count)
    max_count = max(all_count)
    max_index = all_count.index(max_count)
    major = all_tags[max_index]
    minor = all_tags
    minor.remove(major)
    final = [major]+minor
    return final

In [15]:
sim_tags_ordered = []
for tag in squash_list:
    if tag in sim_tags:
        sim_tags_ordered.append(tag)
# print(sim_tags_ordered)
sim_tags_reversed = sim_tags_ordered
sim_tags_reversed.reverse()
print(sim_tags_reversed)
print(len(sim_tags),len(sim_tags_reversed))

['policy', 'illness', 'poverty', 'ecology', 'cognitive science', 'global development', 'genetics', 'mental health', 'philosophy', 'live music', 'biodiversity', 'mind', 'physics', 'potential', 'disease', 'climate change', 'government', 'neuroscience', 'personal growth', 'inequality', 'medical research', 'photography', 'life', 'psychology', 'music', 'health care', 'medicine', 'biology', 'creativity', 'future', 'social change', 'society', 'art', 'innovation', 'health', 'culture', 'global issues', 'science', 'technology']
39 39


In [16]:
final_replacement = {}
for i in range(len(sim_tags_reversed)):
    tag = sim_tags_reversed[i]
    sims = tagsimdict[tag]
    #check if words that main tag is similar too have similar words too
    for sim in sims:
        if sim in final_replacement.keys():
            simlist = final_replacement[sim]
            sims+=simlist
            del final_replacement[sim]
    #remove duplicates
    sims = list(dict.fromkeys(sims))
    final_replacement[tag]=sims
print(final_replacement)

{'biodiversity': ['ecology', 'biology', 'biodiversity'], 'government': ['policy', 'government'], 'inequality': ['poverty', 'inequality'], 'life': ['mind', 'life'], 'music': ['live music', 'music'], 'future': ['potential', 'future'], 'art': ['photography', 'art'], 'culture': ['society', 'culture', 'social change', 'global issues', 'personal growth', 'climate change', 'global development', 'innovation', 'technology', 'creativity'], 'global issues': ['social change', 'climate change', 'global development'], 'science': ['biology', 'psychology', 'physics', 'cognitive science', 'science', 'neuroscience', 'genetics', 'ecology', 'philosophy', 'medical research', 'health', 'medicine', 'health care', 'mental health', 'illness', 'disease'], 'technology': ['innovation']}


In [17]:
print(final_replacement.keys())

dict_keys(['biodiversity', 'government', 'inequality', 'life', 'music', 'future', 'art', 'culture', 'global issues', 'science', 'technology'])


In [18]:
def replacing (x):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in final_replacement.keys():
            #tag is a major
            final_tags.append(tag)
        else:
            #tag is a minor
            r = []
            for k,v in final_replacement.items():
                if tag in v:
                    #tag is a minor w/ a major
                    r.append(k)
            if r == []:
                r.append(tag)
            final_tags += r
    final_tags = list(dict.fromkeys(final_tags))
    final_string = ','.join(final_tags)
    return final_string

In [19]:
alist = 'activism,humanity,identity,personal growth'
b = replacing(alist)
print(b)

activism,humanity,identity,culture


In [20]:
df2 = df1.copy()
df2['squash2_tags'] = df2['squash_tags']
df2['squash2_tags'] = df2['squash2_tags'].map(lambda x: replacing(x))
df2 = df2[df2['squash2_tags']!='']

In [21]:
s_tags = df2['squash2_tags'].str.replace(', ', ',').str.lower().str.strip()
# s_split_tags = s_tags.str.split(',')
# tag_counts_per_talk = s_split_tags.apply(len)

s_joined_tags = s_tags.str.cat(sep=',').split(',')
all_stags_w_dup = pd.Series(s_joined_tags)

s_tag_counts = all_stags_w_dup.value_counts()
s_tag_list = list(s_tag_counts.index)

In [22]:
print(len(s_tag_list))
s_tag_counts

72


culture           1106
science            868
technology         787
global issues      679
design             400
business           329
entertainment      285
art                261
future             218
biodiversity       215
education          206
communication      185
politics           183
humanity           164
collaboration      163
life               156
environment        155
economics          154
brain              148
activism           147
community          136
invention          136
children           135
history            135
inequality         128
music              126
women              115
cities             113
storytelling       112
animals            108
                  ... 
visualizations      80
internet            77
architecture        76
sustainability      74
oceans              73
happiness           71
green               71
biotech             70
work                69
media               66
film                66
violence            65
evolution  

In [23]:
tag_cutoff3 = int(0.05*len(df2.index))

squashed_tags3 = pd.DataFrame(s_tag_counts)
squashed_tags3 = squashed_tags3[(squashed_tags3[0]>tag_cutoff3)]
squash_list3 = list(squashed_tags3.index.values)
print(len(squash_list3))
squashed_tags3

26


,0
culture,1106
science,868
technology,787
global issues,679
design,400
business,329
entertainment,285
art,261
future,218
biodiversity,215


In [24]:
df3 = df2.copy()
df3['squash3_tags'] = df3['squash2_tags']
df3['squash3_tags'] = df3['squash3_tags'].map(lambda x: squashing(x,squash_list3))
df3 = df3[df3['squash3_tags']!='']

In [25]:
tag_cutoff4 = 200

squashed_tags4 = pd.DataFrame(s_tag_counts)
squashed_tags4 = squashed_tags4[(squashed_tags4[0]>tag_cutoff4)]
squash_list4 = list(squashed_tags4.index.values)
print(len(squash_list4))
squashed_tags4

11


,0
culture,1106
science,868
technology,787
global issues,679
design,400
business,329
entertainment,285
art,261
future,218
biodiversity,215


In [26]:
df4 = df3.copy()
df4['squash4_tags'] = df4['squash3_tags']
df4['squash4_tags'] = df4['squash4_tags'].map(lambda x: squashing(x,squash_list4))
df4 = df4[df4['squash4_tags']!='']

In [27]:
tag_cutoff5 = 300

squashed_tags5 = pd.DataFrame(s_tag_counts)
squashed_tags5 = squashed_tags5[(squashed_tags5[0]>tag_cutoff5)]
squash_list5 = list(squashed_tags5.index.values)
print(len(squash_list5))
squashed_tags4

6


,0
culture,1106
science,868
technology,787
global issues,679
design,400
business,329
entertainment,285
art,261
future,218
biodiversity,215


In [28]:
df5 = df4.copy()
df5['squash5_tags'] = df5['squash4_tags']
df5['squash5_tags'] = df5['squash5_tags'].map(lambda x: squashing(x,squash_list4))
df5 = df5[df5['squash4_tags']!='']

In [29]:
df1 = df1.reindex(list(range(len(df1))))
df2 = df2.reindex(list(range(len(df2))))
df3 = df3.reindex(list(range(len(df3))))
df4 = df4.reindex(list(range(len(df4))))
df5 = df5.reindex(list(range(len(df5))))

In [30]:
df1.to_parquet(OUT_DATA_DIR+'cleaned_squashed1.parquet')
df2.to_parquet(OUT_DATA_DIR+'cleaned_squashed2.parquet')
df3.to_parquet(OUT_DATA_DIR+'cleaned_squashed3.parquet')
df4.to_parquet(OUT_DATA_DIR+'cleaned_squashed4.parquet')
df5.to_parquet(OUT_DATA_DIR+'cleaned_squashed4.parquet')

In [31]:
df5

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,squash_tags,squash2_tags,squash3_tags,squash4_tags,squash5_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"culture,politics,science,climate change,enviro...","culture,politics,science,global issues,environ...","culture,politics,science,global issues,environ...","culture,science,global issues,technology","culture,science,global issues,technology"
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"invention,engineering,design,global issues","invention,engineering,design,global issues","invention,design,global issues","design,global issues","design,global issues"
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"poverty,economics,culture,politics,policy,glob...","inequality,economics,culture,politics,governme...","inequality,economics,culture,politics,global i...","culture,global issues,business","culture,global issues,business"
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"invention,engineering,entrepreneur,design,busi...","invention,engineering,entrepreneur,design,busi...","invention,design,business","design,business","design,business"
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"invention,design,technology,business,art","invention,design,technology,business,art","invention,design,technology,business,art","design,technology,business,art","design,technology,business,art"
5,Craig Venter,Sampling the ocean's DNA,Genomics pioneer Craig Venter takes a break fr...,0:16:51,"biotech,invention,oceans,genetics,DNA,biology,...","0:11\r\r\rAt the break, I was asked by several...",2548.0,"[break, ask, people, comment, age, debate, com...",break ask people comment age debate comment un...,"biotech,invention,oceans,genetics,biology,scie...","biotech,invention,oceans,science,biodiversity,...","invention,science,biodiversity,technology","science,biodiversity,technology","science,biodiversity,technology"
6,David Pogue,Simplicity sells,New York Times columnist David Pogue takes aim...,0:21:26,"simplicity,computers,software,interface design...","0:12\r\r\r(Music: ""The Sound of Silence,""\rSim...",3584.0,"[music, sound, silence, simon, garfunkel, hell...",music sound silence simon garfunkel hello voic...,"computers,music,media,entertainment,performanc...","computers,music,media,entertainment,performanc...","music,entertainment,technology","entertainment,technology","entertainment,technology"
7,David Rockwell,A memorial at Ground Zero,In this emotionally charged conversation with ...,0:24:37,"New York,memory,interview,death,culture,archit...","0:13\r\r\rKurt Andersen: Like many architects,...",4008.0,"[kurt, andersen, like, architect, 